https://adventofcode.com/2022/day/12

In [18]:
f = open("input.txt", "r")

s = read(f, String)

data = s

close(f)

In [20]:
mutable struct Node
	elevation
	isVisited
	x
	y
	char
	distance
	previous_node
end

# creates a node object based on the character
function generateNode(char, x, y)
	return Node(getElevation(char), false, x, y, char, Inf, nothing)
end

# creates a grid of nodes and array of the same nodes from the data string
function generateGrid(data)
	row = data |> split
	height = length(row)
	grid = row |> join |> collect
	width = length(grid) ÷ height
	grid = reshape(grid, (width,height))

	nodes = []
	node_grid = Matrix(undef, width, height)

	for x in 1:width, y in 1:height
		node = generateNode(grid[x, y], x, y)
		node_grid[x, y] = node
		push!(nodes, node)
	end

	return (node_grid, nodes)
end

# calculates the elevation with 'a' = 0
function getElevation(char)
	return char - 'a'
end

# does a breath first search of the grid from start to finish nodes
function search(start, finish, grid, nodes)	
	start.distance = 0 # sets the start node to 0

	# loops through all the nodes in order by the shorted distance from the start
	while length(nodes) > 0

		# visits the closest node from the previous node that hasn't been visited before
		sort!(nodes,by = node -> node.distance)
		node = popfirst!(nodes)
		node.isVisited = true

		# the node is unreachable if it's distance is infinite
		if node.distance == Inf
			println("no path found")
			return missing
		end

		# if the node is the finish node return it
		if node == finish
			return node
		end

		# updates the node's neighbors distance from it if visitable
		update_neighbors(node, grid)
	end
end

function update_neighbors(node, grid)
	neighbors = get_neighbors(node, grid)

	for neighbor in neighbors
		neighbor.distance = node.distance + 1
		neighbor.previous_node = node
	end

end

function get_neighbors(node, grid)
	x = node.x
	y = node.y

	width, height = size(grid)

	neighbors = []

	# node right
	if x > 1
		neighbor = grid[x-1, y]
		if isVisitable(node, neighbor)
			push!(neighbors, grid[x-1, y])
		end
	end

	# node below
	if y > 1
		neighbor = grid[x, y-1]
		if isVisitable(node, neighbor)
			push!(neighbors, grid[x, y-1])
		end
	end

	# node left
	if x < width
		neighbor = grid[x+1, y]
		if isVisitable(node, neighbor)
			push!(neighbors, grid[x+1, y])
		end
	end

	# node above
	if y < height
		neighbor = grid[x, y+1]
		if isVisitable(node, neighbor)
			push!(neighbors, grid[x, y+1])
		end
	end

	return neighbors
end

# checks if the node has been visited before and if it's the proper elevation to climb
function isVisitable(node, neighbor)
	return neighbor.elevation <= node.elevation+1 && !neighbor.isVisited
end

# returns the shortest path as an array of nodes from finish to start
function getPath(data)
	grid, nodes = generateGrid(data)

	start_node_index = findfirst(node -> node.char == 'S', nodes)
	start_node = nodes[start_node_index]
	start_node.elevation = getElevation('a')

	finish_node_index = findfirst(node -> node.char == 'E', nodes)
	finish_node = nodes[finish_node_index]
	finish_node.elevation = getElevation('z')
	
	node = search(start_node, finish_node, grid, nodes)

	path = []

	while node.previous_node != nothing
		node = node.previous_node
		push!(path,node)
	end


	return path
end

function getShortedPathLength(data)
	return data |> getPath |> length
end

getShortedPathLength(data)


534

In [25]:
# searches from start until it finds a node with elevation 0
function search2(start, grid, nodes)	
	start.distance = 0
	path = []

	while length(nodes) > 0
		sort!(nodes,by = node -> node.distance)
		node_closest = popfirst!(nodes)
		node_closest.isVisited = true

		push!(path, node_closest)

		if node_closest.distance == Inf
			println("no path found")
			return missing
		end

		if node_closest.elevation == 0
			return node_closest
		end

		update_neighbors2(node_closest, grid)
	end
end

function update_neighbors2(node, grid)
	neighbors = get_neighbors2(node, grid)

	for neighbor in neighbors
		neighbor.distance = node.distance + 1
		neighbor.previous_node = node
	end

end

function get_neighbors2(node, grid)
	x = node.x
	y = node.y

	width, height = size(grid)

	neighbors = []

	if x > 1
		neighbor = grid[x-1, y]
		if isVisitable2(node, neighbor)
			push!(neighbors, grid[x-1, y])
		end
	end

	if y > 1
		neighbor = grid[x, y-1]
		if isVisitable(node, neighbor)
			push!(neighbors, grid[x, y-1])
		end
	end

	if x < width
		neighbor = grid[x+1, y]
		if isVisitable(node, neighbor)
			push!(neighbors, grid[x+1, y])
		end
	end

	if y < height
		neighbor = grid[x, y+1]
		if isVisitable(node, neighbor)
			push!(neighbors, grid[x, y+1])
		end
	end

	return neighbors
end

# changes the if the node is visitable going down in elevation instead of up
function isVisitable2(node, neighbor)
	return neighbor.elevation >= node.elevation-1 && !neighbor.isVisited
end

# gets path from 'E' to node of elevation 0
function getPath2(data)
	grid, nodes = generateGrid(data)

	start_node_index = findfirst(node -> node.char == 'E', nodes)
	start_node = nodes[start_node_index]
	start_node.elevation = getElevation('z')
	
	node = search2(start_node, grid, nodes)

	path = []

	while node.previous_node != nothing
		node = node.previous_node
		push!(path,node)
	end


	return path
end

function getShortedPathLength2(data)
	return data |> getPath2 |> length
end

getShortedPathLength2(data)

525